# Implicitly defined reachable sets within GCS

In [ ]:
import numpy as np
from large_gcs.graph_generators.hor_vert_gcs import create_simplest_hor_vert_graph, create_intermediate_line_hor_vert_graph

# from explorations.hor_vert_gcs_bug import create_simplest_hor_vert_graph

from pydrake.all import (MathematicalProgram, MathematicalProgramResult, Solve)

import logging

logging.basicConfig(level=logging.WARN)
logging.getLogger("large_gcs").setLevel(logging.DEBUG)

## Simple Example Graph

In [ ]:
# g = create_simplest_hor_vert_graph()
g = create_intermediate_line_hor_vert_graph()
g.plot_sets()
g.plot_edges()
g.plot_set_labels()

In [ ]:
sol = g.solve_shortest_path()
g.plot_sets()
g.plot_edges()
g.plot_path(sol.ambient_path)
sol

In [ ]:
print(g.edge_keys)
# sol_conv_res = g.solve_convex_restriction([("s", "p1"), ("p1", "p2"), ("p2", "t")])

# sol_conv_res
# g.plot_sets()
# g.plot_path(sol_conv_res.ambient_path)

In [ ]:
for vertex_name, v in g.vertices.items():
    print(vertex_name, v.gcs_vertex.set().IsBounded())

# Sampling Procedure
To determine if A is dominated by B

In [ ]:
# Sample 100 points in p2
# p2_samples = np.array(g.vertices["p2"].convex_set.get_samples(100))
p3_samples = np.array(g.vertices["p3"].convex_set.get_samples(20))
g.plot_sets()
g.plot_points(p3_samples)

In [ ]:
# Project these samples into revised set based on path


def project_sample_to_revised_set(samples, vertex_sampled, active_edges):
    prog = MathematicalProgram()
    vertex_names = [g.edges[active_edges[0]].u] + [g.edges[e].v for e in active_edges]
    # gcs vertices
    vertices = [g.vertices[name].gcs_vertex for name in vertex_names]
    edges = [g.edges[edge].gcs_edge for edge in active_edges]
    vertex_vars = [prog.NewContinuousVariables(rows=samples.shape[0], cols=v.ambient_dimension(), name=f"{v_name}_vars") for v, v_name in zip(vertices, vertex_names)]
    sample_vars = vertex_vars[vertex_names.index(vertex_sampled)]
    for v, v_name, v_vars in zip(vertices, vertex_names, vertex_vars):
        if v_name == vertex_sampled:
            for x, sample in zip(v_vars, samples):
                v.set().AddPointInSetConstraints(prog, x)
                # Add the distance to the sample as a cost
                prog.AddCost((x - sample).dot(x - sample))
                # Vertex Constraints
                for binding in v.GetConstraints():
                    constraint = binding.evaluator()
                    prog.AddConstraint(constraint, x)
        else:
            for x in v_vars:
                v.set().AddPointInSetConstraints(prog, x)

                # Vertex Constraints
                for binding in v.GetConstraints():
                    constraint = binding.evaluator()
                    prog.AddConstraint(constraint, x)

    for e, e_name in zip(edges, active_edges):
        # Edge Costs
        # for binding in e.GetCosts():
        #     cost = binding.evaluator()
        #     variables = binding.variables()
        #     prog.AddCost(cost, variables)
        
        # Edge Constraints
        for binding in e.GetConstraints():
            constraint = binding.evaluator()
            variables = binding.variables()
            u_name, v_name = g.edges[e_name].u, g.edges[e_name].v
            u_idx, v_idx = vertex_names.index(u_name), vertex_names.index(v_name)
            for ux, vx in zip(vertex_vars[u_idx], vertex_vars[v_idx]):
                variables[:len(ux)] = ux
                variables[-len(vx):] = vx
                prog.AddConstraint(constraint, variables)

    result = Solve(prog)
    if not result.is_success():
        raise RuntimeError("Failed to solve")
    return result.GetSolution(sample_vars)


# active_edges = ["('s', 'p1')", "('p1', 'p2')"]
active_edges = ["('s', 'p1')", "('p1', 'p2')", "('p2', 'p3')"]
# active_edges = ["('s', 'p1')", "('p1', 'p2')", "('p2', 't')"]
new_samples = project_sample_to_revised_set(p3_samples, "p3", active_edges)
# print(new_samples)
g.plot_points(new_samples)
g.plot_sets()

# GcsAstarReachability

In [ ]:
from large_gcs.algorithms.gcs_astar_reachability import GcsAstarReachability
from large_gcs.cost_estimators.shortcut_edge_ce import ShortcutEdgeCE
from large_gcs.graph.contact_cost_constraint_factory import (
    contact_shortcut_edge_cost_factory_over_obj_weighted,
)
from large_gcs.graph.cost_constraint_factory import (
    shortcut_edge_cost_factory,
)
import numpy as np

from large_gcs.graph_generators.hor_vert_gcs import create_simplest_hor_vert_graph, create_polyhedral_hor_vert_graph, create_intermediate_line_hor_vert_graph
# from explorations.hor_vert_gcs_bug import create_simplest_hor_vert_graph

import logging

logging.basicConfig(level=logging.WARN)
logging.getLogger("large_gcs").setLevel(logging.DEBUG)

In [ ]:
# g = create_simplest_hor_vert_graph()
g = create_intermediate_line_hor_vert_graph()
# g = create_polyhedral_hor_vert_graph()
cost_estimator_se = ShortcutEdgeCE(g, shortcut_edge_cost_factory)
alg = GcsAstarReachability(g, cost_estimator_se, num_samples_per_vertex=5)
sol = alg.run()

In [ ]:
# g.edges["('p0', 'p0_sample_1')"]

In [ ]:
g.plot_sets()
g.plot_edges()
g.plot_set_labels()
g.plot_path(sol.ambient_path)

In [ ]:
# sol = g.solve_convex_restriction([('s', 'p1'), ('p1', 'p1_sample_0')])
# sol = g.solve_convex_restriction([("s", "p1"), ("p1", "p2"), ("p2", "t")])

In [ ]:
from large_gcs.algorithms.gcs_astar_convex_restriction import GcsAstarConvexRestriction

g = create_polyhedral_hor_vert_graph()
cost_estimator_se = ShortcutEdgeCE(g, shortcut_edge_cost_factory)
alg = GcsAstarConvexRestriction(g, cost_estimator_se)
sol = alg.run()

Note how GcsAstarConvexRestriction fails because it doesn't have the sampling to determine reachability procedure that GcsAstarReachability has

# Applying GcsAstarReachability to Contact Graph that fails for GcsAstarConvexRestriction
cg_maze_b1

In [3]:
from large_gcs.algorithms.gcs_astar_reachability import GcsAstarReachability
from large_gcs.cost_estimators.shortcut_edge_ce import ShortcutEdgeCE
from large_gcs.graph.contact_cost_constraint_factory import (
    contact_shortcut_edge_cost_factory_over_obj_weighted,
)
from large_gcs.graph.cost_constraint_factory import (
    shortcut_edge_cost_factory,
)
from large_gcs.algorithms.search_algorithm import SearchNode
from large_gcs.graph.graph import ShortestPathSolution
from large_gcs.graph.incremental_contact_graph import IncrementalContactGraph
from large_gcs.graph_generators.contact_graph_generator import ContactGraphGeneratorParams

import numpy as np
from IPython.display import HTML

import logging

logging.basicConfig(level=logging.WARN)
logging.getLogger("large_gcs").setLevel(logging.DEBUG)
logging.getLogger(__name__).setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)

In [4]:
graph_file = ContactGraphGeneratorParams.inc_graph_file_path_from_name("cg_maze_b1")
cg = IncrementalContactGraph.load_from_file(
    graph_file,
    should_incl_simul_mode_switches=False,
    should_add_const_edge_cost=True,
    should_add_gcs=True,
)
logger.info("Loaded graph\n\n")

contact_pair_mode_ids = ('IC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f1-rob0_f1')
contact_set = cg._create_contact_set_from_contact_pair_mode_ids(contact_pair_mode_ids)

# vertex_path = ['source',
#  "('NC|obs0_f0-obj0_v3', 'NC|obs0_f1-rob0_v0', 'NC|obj0_f2-rob0_f2')",
#  "('NC|obs0_f0-obj0_v3', 'NC|obs0_v1-rob0_f1', 'NC|obj0_f2-rob0_f2')",
#  "('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f2-rob0_f2')",
#  "('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')",
#  "('IC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')",
#  "('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')"]

# for v in vertex_path:
#     print(f"Generating neighbors for {v}")
#     cg.generate_neighbors(v)



INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 2/2 [00:00<00:00, 34952.53it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 18641.35it/s]
INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...
100%|██████████| 3/3 [00:00<00:00, 50.33it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (151)
100%|██████████| 151/151 [00:00<00:00, 550.24it/s]
DEBUG:large_gcs.geometry.polyhedron:
 A.shape: (50, 14), b.shape: (50,), C.shape: (8, 14), d.shape: (8,)
DEBUG:large_gcs.geometry.polyhedron:ranks: A: 14, C: 8
DEBUG:large_gcs.geometry.polyhedron:C = np.array([[1.,-1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.],[0.,0.,1.,-1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.],[0.,0.,0.,0.,1.,-1.,0.,0.,0.,0.,1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,1.,-1.,0.,0.,0.,1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1

In [ ]:
from large_gcs.utils.utils import copy_pastable_str_from_np_array


print(copy_pastable_str_from_np_array(contact_set.set.A()))
print(copy_pastable_str_from_np_array(contact_set.set.b()))

In [ ]:
from large_gcs.geometry.polyhedron import Polyhedron


A = np.array([[0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,-1.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,1.,0.],[0.,0.,0.,0.,-0.78374588,0.,0.62108164,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,-0.78374588,0.,0.62108164,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[-1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[1.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,-1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,1.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,-1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,-1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,1.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.],[1.,-1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[-1.,1.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,1.,-1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,-1.,1.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.],[0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,-1.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,-1.,0.,0.,0.],[0.,0.,0.,0.,1.,-1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,-1.,1.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,1.,-1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,-1.,1.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,-1.,0.,0.,-1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,1.,0.,0.,1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,-1.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,1.,0.,0.,0.,0.],[1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.],[-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-1.]])
b = np.array([4.32100000e+00,-4.32100000e+00,4.32100000e+00,-4.32100000e+00,-1.86000000e+00,2.86000000e+00,-1.86000000e+00,2.86000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-1.06820936e+00,-1.06820936e+00,6.66682241e-01,-6.66682241e-01,6.66682241e-01,-6.66682241e-01,8.33324081e-01,1.16667592e+00,8.33324081e-01,1.16667592e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,-0.00000000e+00,4.50000000e+00,4.50000000e+00,4.50000000e+00,4.50000000e+00,4.50000000e+00,4.50000000e+00,4.50000000e+00,4.50000000e+00,4.66668224e+00,4.33332408e+00,4.83331776e+00,4.66667592e+00,4.66668224e+00,4.33332408e+00,4.83331776e+00,4.66667592e+00,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03,1.00000000e+03])
P = Polyhedron(A, b)
P.get_samples(5)

In [5]:
contact_set.get_samples(5)

ValueError: The Hit and Run algorithm failed to find a feasible point in the set. The `previous_sample` must be in the set.
max(A * previous_sample - b) = 5.411712424021244e-16

In [ ]:
convex_set = cg.vertices["('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')"].convex_set
convex_set.get_samples(5)

In [37]:
convex_set = contact_set
convex_set.set.IsBounded(), convex_set.set.IsEmpty(), convex_set._polyhedron._null_space_polyhedron.set.IsBounded(), convex_set._polyhedron._null_space_polyhedron.set.IsEmpty()
# prev_sample = convex_set._polyhedron._null_space_polyhedron.set.ChebyshevCenter()
# set = convex_set._polyhedron._null_space_polyhedron.set
# A = set.A()
# b = set.b().reshape(-1, 1)
# prev_sample = prev_sample.reshape(-1, 1)
# violation = A @ prev_sample - b
# # Find any rows greater or equal to 0
# violating_rows = violation[np.where(violation >= 0)]
# violating_rows

# # prev_sample = set.MaybeGetFeasiblePoint()
# set.PointInSet(prev_sample,)
# # set.PointInSet(prev_sample, tol=0)

# # A.shape
# # prev_sample.shape
# # convex_set.get_samples(5)
# A.shape, b.shape

(True, False, True, False)

In [10]:
convex_set._polyhedron._null_space_polyhedron._has_equality_constraints, convex_set._polyhedron._null_space_polyhedron._null_space_polyhedron.set.IsEmpty()

(False,)

In [2]:


# cg_maze_b1_1 cg_maze_b1
graph_file = ContactGraphGeneratorParams.inc_graph_file_path_from_name("cg_maze_b1")
cg = IncrementalContactGraph.load_from_file(
    graph_file,
    should_incl_simul_mode_switches=False,
    should_add_const_edge_cost=True,
    should_add_gcs=True,
)
cost_estimator = ShortcutEdgeCE(
    cg,
    shortcut_edge_cost_factory=contact_shortcut_edge_cost_factory_over_obj_weighted,
)
alg = GcsAstarReachability(cg, cost_estimator, num_samples_per_vertex=5)
sol: ShortestPathSolution = alg.run()


INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 2/2 [00:00<00:00, 11244.78it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 9765.55it/s]
INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...
100%|██████████| 3/3 [00:00<00:00, 44.28it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (151)
100%|██████████| 151/151 [00:00<00:00, 566.35it/s]
DEBUG:large_gcs.geometry.polyhedron:
 A.shape: (50, 14), b.shape: (50,), C.shape: (8, 14), d.shape: (8,)
DEBUG:large_gcs.geometry.polyhedron:ranks: A: 14, C: 8
DEBUG:large_gcs.geometry.polyhedron:C = np.array([[1.,-1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.],[0.,0.,1.,-1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.],[0.,0.,0.,0.,1.,-1.,0.,0.,0.,0.,1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,1.,-1.,0.,0.,0.,1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.

ValueError: The Hit and Run algorithm failed to find a feasible point in the set. The `previous_sample` must be in the set.
max(A * previous_sample - b) = 5.411712424021244e-16

In [ ]:
# cg.vertices["('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')"].convex_set._null_space_polyhedron.set.IsEmpty()
# cg.vertices["('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')"].convex_set._polyhedron._null_space_polyhedron.set.IsEmpty()
convex_set = cg.vertices["('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')"].convex_set
convex_set.get_samples(5)
# n = alg._S["('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')"][0]
# n.vertex_path

In [ ]:
anim = cg.animate_solution()
HTML(anim.to_html5_video())


In [ ]:

v_path = ['source', "('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f0-rob0_v1')", "('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')", "('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f0-rob0_v1')"]
n = SearchNode(priority=544.4999870205717, 
                  vertex_name="('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f0-rob0_v1')", 
                  edge_path=[
                      '(\'source\', "(\'NC|obs0_f1-obj0_v1\', \'NC|obs0_f0-rob0_v1\', \'NC|obj0_f0-rob0_v1\')")', 
                      '("(\'NC|obs0_f1-obj0_v1\', \'NC|obs0_f0-rob0_v1\', \'NC|obj0_f0-rob0_v1\')", "(\'NC|obs0_f1-obj0_v1\', \'NC|obs0_f0-rob0_v1\', \'IC|obj0_f1-rob0_f1\')")', 
                      '("(\'NC|obs0_f1-obj0_v1\', \'NC|obs0_f0-rob0_v1\', \'IC|obj0_f1-rob0_f1\')", "(\'NC|obs0_f1-obj0_v1\', \'NC|obs0_f0-rob0_v1\', \'IC|obj0_f0-rob0_v1\')")'
                      ], 
                vertex_path=[
                    'source', 
                    "('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f0-rob0_v1')", 
                    "('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')", 
                    "('NC|obs0_f1-obj0_v1', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f0-rob0_v1')"
                    ]
                )
# alg._graph.generate_and_plot_samples_in_set(n.vertex_name, n_samples=1)
# alg._graph.set_target(n.vertex_name)
# alg._graph.solve_convex_restriction(n.edge_path)
# alg._graph.plot_path()
# anim = alg._graph.animate_solution()
# HTML(anim.to_html5_video())
# alg._graph.plot()
proj_samples = alg._set_samples[n.vertex_name].project(alg._graph, n)
alg._graph.plot_samples_in_set(n.vertex_name, proj_samples)


The equality constraints on edges can collapse the feasible region in the next set to some measure 0 subspace which then have probability 0 of being sampled